# Feature-Engineering

Do your work for this exercise in a jupyter notebook named feature_engineering within the regression-exercises repo. Add, commit, and push your work.

In [1]:
from pydataset import data
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
import sklearn.linear_model
import sklearn.feature_selection
import sklearn.preprocessing

## 1. 

## Acquire
- Load the tips dataset.

In [2]:
df = data('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.shape

(244, 7)

## Prepare

### a. 
- Create a column named price_per_person. 
- This should be the total bill divided by the party size.

In [4]:
# rename size column because it is a Pandas attribute
df = df.rename(columns={'size': 'group_size'})
df.head()

,total_bill,tip,sex,smoker,day,time,group_size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [5]:
df['price_per_person'] = df.total_bill / df.group_size
df.head()

,total_bill,tip,sex,smoker,day,time,group_size,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,6.147500


### b. 
- Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

To predict tips in the future, I think the best method would be to look at the average amount tipped in comparrison to the total bill (otherwise known as the tip percentage). To do this, we will need to add a tip percentage to the dataframe.

In [6]:
df['tip_percentage'] = df.tip / df.total_bill
df.head()

,total_bill,tip,sex,smoker,day,time,group_size,price_per_person,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,8.495000,0.059447
2,10.34,1.66,Male,No,Sun,Dinner,3,3.446667,0.160542
3,21.01,3.50,Male,No,Sun,Dinner,3,7.003333,0.166587
4,23.68,3.31,Male,No,Sun,Dinner,2,11.840000,0.139780
5,24.59,3.61,Female,No,Sun,Dinner,4,6.147500,0.146808


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 1 to 244
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_bill        244 non-null    float64
 1   tip               244 non-null    float64
 2   sex               244 non-null    object 
 3   smoker            244 non-null    object 
 4   day               244 non-null    object 
 5   time              244 non-null    object 
 6   group_size        244 non-null    int64  
 7   price_per_person  244 non-null    float64
 8   tip_percentage    244 non-null    float64
dtypes: float64(4), int64(1), object(4)
memory usage: 19.1+ KB


-- I will only need the numeric values for this.

In [8]:
# Make dataframe with only numeric values
df = df[['total_bill', 'tip', 'group_size', 'price_per_person', 'tip_percentage']]
df.head()

,total_bill,tip,group_size,price_per_person,tip_percentage
1,16.99,1.01,2,8.495000,0.059447
2,10.34,1.66,3,3.446667,0.160542
3,21.01,3.50,3,7.003333,0.166587
4,23.68,3.31,2,11.840000,0.139780
5,24.59,3.61,4,6.147500,0.146808


### Prepare - Split

### c. 
- Use select k best to select the top 2 features for predicting tip amount. What are they?

In [9]:
# split the data before feature engineering
def split(df):
    '''
    take in a DataFrame and return train, validate, and test DataFrames.
    return train, validate, test DataFrames.
    '''
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)
    train, validate = train_test_split(train_validate, 
                                       test_size=.3, 
                                       random_state=123)
    return train, validate, test

In [10]:
train, validate, test = split(df)
train.shape, validate.shape, test.shape

((136, 5), (59, 5), (49, 5))

### Prepare - Set X & y
- X (Predicters  / Independent Variables) & y (Target / Dependent Variable)

In [11]:
target = "tip"

# split train into X (dataframe, drop target) & y (series, keep target only)
X_train = train.drop(columns=[target])
y_train = train[target]

# split validate into X (dataframe, drop target) & y (series, keep target only)
X_validate = validate.drop(columns=[target])
y_validate = validate[target]

# split test into X (dataframe, drop target) & y (series, keep target only)
X_test = test.drop(columns=[target])
y_test = test[target]

X_train.head()

,total_bill,group_size,price_per_person,tip_percentage
19,16.97,3,5.656667,0.206246
173,7.25,2,3.625000,0.710345
119,12.43,2,6.215000,0.144811
29,21.70,2,10.850000,0.198157
238,32.83,2,16.415000,0.035638


## Model - Feature Engineering

### K-Best

In [12]:
# Apply Select K Best filter method

from sklearn.feature_selection import SelectKBest, f_regression

In [13]:
# Make the filter
kbest = SelectKBest(f_regression, k=2)

# Fit the filter
kbest.fit(X_train, y_train)

# Obtain results as a dataframe
kbest_results = pd.DataFrame(dict(p=kbest.pvalues_, f=kbest.scores_), index=X_train.columns)
kbest_results

,p,f
total_bill,7.186470e-20,115.984909
group_size,1.341642e-12,61.259089
price_per_person,1.310327e-03,10.777792
tip_percentage,7.636450e-06,21.686627


In [14]:
# Obtain list of top 2 features to predict tip amount
X_train.columns[kbest.get_support()]

Index(['total_bill', 'group_size'], dtype='object')

In [15]:
X_train_transformed = pd.DataFrame(
    kbest.transform(X_train),
    index=X_train.index,
    columns=X_train.columns[kbest.get_support()]
)
X_train_transformed.head().sort_index()

,total_bill,group_size
19,16.97,3.0
29,21.70,2.0
119,12.43,2.0
173,7.25,2.0
238,32.83,2.0


### RFE

### d. 
- Use recursive feature elimination to select the top 2 features for tip amount. What are they?

### Scale the Data

- Scale the Data for Modeling with Continuous Data

In [16]:

### Go back and fix dataframes

def scaled_data(X_train, X_validate, X_test, continuous_cols):

    scaler = sklearn.preprocessing.MinMaxScaler()

    # Fit the scaler, (fit the thing)
    scaler.fit(X_train[continuous_cols])

    # Use the scaler to transform train, validate, test (use the thing)
    X_train_scaled = pd.DataFrame(scaler.transform(X_train[continuous_cols])
    X_validate_scaled = pd.DataFrame(scaler.transform(X_validate[continuous_cols])
    X_test_scaled = pd.DataFrame.scaler(transform(X_test[continuous_cols])
    
    return X_train_scaled, X_validate_scaled, X_test_scaled 

In [35]:
scaled_data(X_train, X_validate, X_test,['total_bill', 'group_size', 'price_per_person', 'tip_percentage'])

KeyError: "None of [Index(['total_bill', 'group_size', 'price_per_person', 'tip_percentage'], dtype='object')] are in the [columns]"

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

In [19]:
model = LinearRegression()
# Make the thing
rfe = RFE(model, n_features_to_select=2)
# Fit the thing
rfe.fit(X_train, y_train)
# Obtain results in dataframe
pd.DataFrame({'rfe_ranking': rfe.ranking_}, index=X_train.columns).sort_values(by = 'rfe_ranking')

,rfe_ranking
group_size,1
tip_percentage,1
total_bill,2
price_per_person,3


In [20]:
X_train.columns[rfe.get_support()]

Index(['group_size', 'tip_percentage'], dtype='object')

In [21]:
X_train_transformed = pd.DataFrame(
    rfe.transform(X_train),
    index=X_train.index,
    columns=X_train.columns[rfe.support_]
)
X_train_transformed.head()

,group_size,tip_percentage
19,3.0,0.206246
173,2.0,0.710345
119,2.0,0.144811
29,2.0,0.198157
238,2.0,0.035638


### e. 
- Why do you think select k best and recursive feature elimination might give different answers for the top features? 



Kbest looks at each feature in isolation
RFE looks at multiple features - disentangle correclated independent variables

-Example - exam score - coffee consumption and hours of sleep
K-Best 
RFE - what the model thinks are the most predictive feature determins what to keep or drop




- Does this change as you change the number of features your are selecting?

## 2. K-Best Function

Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [22]:
def select_kbest(X, y, k):
      
    # Make the filter
    kbest = SelectKBest(f_regression, k=k)

    # Fit the filter (may need to call with X_train, Y_train or other)
    kbest.fit(X, y)
    
    # Obtain list of top k features to predict tip amount
    return X_train.columns[kbest.get_support()]


In [23]:
select_kbest(X_train, y_train, 2)

Index(['total_bill', 'group_size'], dtype='object')

## 3. rfe Function

Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [24]:
def rfe(X, y, n):
    
    model = LinearRegression()
    
    # Make the thing
    rfe = RFE(model, n_features_to_select = n)
    # Fit the thing
    rfe.fit(X, y)
    # Obtain results in
    return X_train.columns[rfe.get_support()]

In [25]:
rfe(X_train, y_train, 2)

Index(['group_size', 'tip_percentage'], dtype='object')

## 4. Swiss Dataset

Load the swiss dataset and use all the other features to 
- predict Fertility. 
Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [26]:
df = data('swiss')
df.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, Courtelary to Rive Gauche
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fertility         47 non-null     float64
 1   Agriculture       47 non-null     float64
 2   Examination       47 non-null     int64  
 3   Education         47 non-null     int64  
 4   Catholic          47 non-null     float64
 5   Infant.Mortality  47 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 2.6+ KB


In [28]:
train, validate, test = split(df)

In [29]:
train.shape, validate.shape, test.shape

((25, 6), (12, 6), (10, 6))

In [30]:
target = "Fertility"

# split train into X (dataframe, drop target) & y (series, keep target only)
X_train = train.drop(columns=[target])
y_train = train[target]

# split validate into X (dataframe, drop target) & y (series, keep target only)
X_validate = validate.drop(columns=[target])
y_validate = validate[target]

# split test into X (dataframe, drop target) & y (series, keep target only)
X_test = test.drop(columns=[target])
y_test = test[target]

X_train.head()

,Agriculture,Examination,Education,Catholic,Infant.Mortality
Rolle,60.8,16,10,7.72,16.3
Lavaux,73.0,19,9,2.84,20.0
Nyone,50.9,22,12,15.14,16.7
Conthey,85.9,3,2,99.71,15.1
Yverdon,49.5,15,8,6.10,22.5


### Scale

In [31]:
def scaled_data(X_train, X_validate, X_test, continuous_cols):

    scaler = sklearn.preprocessing.MinMaxScaler()

    # Fit the scaler, (fit the thing)
    scaler.fit(X_train[continuous_cols])

    # Use the scaler to transform train, validate, test (use the thing)
    X_train_scaled = scaler.transform(X_train[continuous_cols])
    X_validate_scaled = scaler.transform(X_validate[continuous_cols])
    X_test_scaled = scaler.transform(X_test[continuous_cols])
    
    return X_train_scaled, X_validate_scaled, X_test_scaled 

In [32]:
# scaled_data(X_train, X_validate, X_test,[''])

### Feature Engineering (K-Best) 

In [33]:
select_kbest(X_train, y_train, 3)

Index(['Examination', 'Catholic', 'Infant.Mortality'], dtype='object')

### Feature Engineering (K-Best) 

In [34]:
rfe(X_train, y_train, 2)

Index(['Examination', 'Infant.Mortality'], dtype='object')